In [15]:
import warnings
import pyart 
import proplot as plot
import numpy as np
import leroi
import time

In [6]:
#read radar data
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    radar = pyart.aux_io.read_odim_h5('/home/564/jb2354/dev/leroi/notebook/dev/50_20200915_004154.pvol.h5', 
                                      file_field_names=True)
#mask invalid
radar = leroi.mask_invalid_data(radar, 'DBZH', min_field = 100, min_area = 50, return_smooth = False)

dict_keys(['DBZH'])


In [31]:
#configure gridding
gs = (25, 301, 301)
gb = ((500, 12000), (-150000,150000),(-150000,150000))
center_pos = (0,0,0)
x = np.linspace(gb[2][0],gb[2][1], gs[2])
y = np.linspace(gb[1][0],gb[1][1], gs[1])
z = np.linspace(gb[0][0],gb[0][1], gs[0])
coords = (z-center_pos[0], y-center_pos[1], x-center_pos[2])

In [26]:
def cressman_ppi_interp(
    radar,
    coords,
    field_names=None,
    gatefilter=None,
    Rc=None,
    k=100,
    filter_its=0,
    verbose=True,
    kernel=None,
    corr_lens=None,
    multiprocessing=True,
    ground_elevation=-999,
):

    t0 = time.time()

    if field_names is None:
        # No field defined. Processing all fields in radar.
        field_names = [*radar.fields.keys()]
    if type(field_names) != list:
        field_names = [field_names]

    fields = {}
    dims = [len(coord) for coord in coords]

    if Rc is None:
        Rc = leroi.leroi.get_leroy_roi(radar, coords)
        if verbose:
            print("Radius of influence set to {} m.".format(Rc))

    dmask = leroi.leroi.get_data_mask(radar, field_names)
    weights, idxs, model_idxs, sw, model_lens = leroi.leroi._setup_interpolate(
        radar, coords, dmask, Rc, multiprocessing, k, verbose
    )
    Z, Y, X = np.meshgrid(coords[0], coords[1], coords[2], indexing="ij")
    ppi_height = leroi.leroi._calculate_ppi_heights(radar, coords, Rc, multiprocessing, ground_elevation)

    if ppi_height.mask.sum() > 0:
        warnings.warn(
            """\n There are invalid height values which will
        ruin the linear interpolation. This most likely means the radar
        doesnt cover the entire gridded domain"""
        )

    elevations = radar.fixed_angle["data"]

    # sort sweep index to process from lowest sweep and ascend
    sort_idx = list(np.argsort(elevations))
    if 90.0 in elevations:
        sort_idx.remove(np.argwhere(elevations == 90))

    for field in field_names:
        ppis = np.zeros((radar.nsweeps, dims[1] * dims[2]))
        mask = np.ones((radar.nsweeps, dims[1] * dims[2]))
        for i, j in enumerate(sort_idx):
            slc = radar.get_slice(j)
            data = radar.fields[field]["data"].filled(0)[slc][~dmask[slc]]
            if len(data) > 0:
                ppis[i, model_idxs[i, : model_lens[i]]] = (
                    np.sum(data[idxs[i]] * weights[i], axis=1) / sw[i, model_idxs[i, : model_lens[i]]]
                )
                mask[i, model_idxs[i, : model_lens[i]]] = 0
                out = np.ma.masked_array(
                    ppis.reshape((radar.nsweeps, dims[1], dims[2])), mask.reshape((radar.nsweeps, dims[1], dims[2]))
                )
            else:
                out = np.ma.masked_array(np.zeros((radar.nsweeps, dims[1], dims[2])), 
                                         np.ones((radar.nsweeps, dims[1], dims[2])))

        grid = leroi.leroi.interp_along_axis(out.filled(np.nan), ppi_height, Z, axis=0, method="linear")

        if filter_its > 0:
            grid = leroi.leroi.smooth_grid(grid, coords, kernel, corr_lens, filter_its, verbose)

        # add to output dictionary
        fields[field] = {"data": np.ma.masked_array(grid, mask=np.isnan(grid))}
        # copy the metadata from the radar to the grid
        for key in radar.fields[field].keys():
            if key == "data":
                continue
            fields[field][key] = radar.fields[field][key]

    if verbose:
        print("Took: ", time.time() - t0)

    return fields

In [39]:
fields = cressman_ppi_interp(radar, coords, ['DBZH'], verbose=False)
grid = leroi.build_pyart_grid(radar, fields, gs, gb)

/home/564/jb2354/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: 
 There are invalid height values which will
        ruin the linear interpolation. This most likely means the radar
        doesnt cover the entire gridded domain


In [41]:
grid.fields['DBZH']

{'data': masked_array(
   data=[[[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],
 
         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],
 
         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],
 
         ...,
 
         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --,

## Maybe a new problem

In [35]:
Rc = leroi.leroi.get_leroy_roi(radar, coords)
ppi_height = leroi.leroi._calculate_ppi_heights(radar, coords, Rc, True, 1)
ppi_height.mask.sum()

14

In [37]:
np.where(ppi_height.mask==True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 array([150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150,
        150]),
 array([150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150, 150,
        150]))

In [38]:
ppi_height.shape

(15, 301, 301)